# Upsert AOOS Priority Score Demo

## Approaching Out of Stock (AOOS)

* Priority scores of work items (inventories) in AOOS work queue are calculated and upserted to InfoHub
* The function `AOOS_priority_score` is defined below - for understanding the business logic, refer to the accompanying Notebook **AOOS-Priority-Score.ipynb**

## InfoHub

* The InfoHub connection and queries are defined in the accompanying Notebook **InfoHub.ipynb**
* Make sure that you have run the Kernel of the above Notebook

In [ ]:
import json
import sys
import os
import pandas as pd
import time
import numpy as np
import datetime
import copy
import json

### Import `InfoHubConnection` Class

* Install `ipynb` package for the following import to work

    `pip install ipynb`
    
* Make sure that the Kernel of Notebook **InfoHub.ipynb** has been run without errors

In [ ]:
from ipynb.fs.full.InfoHub import InfoHubConnection

### Priority Score Calculation

* Priority score function for _Approaching Out of Stock_ work item
* The business logic is explained in detail in the accompanying Notebook **AOOS-Priority-Score.ipynb**

In [ ]:
def AOOS_priority_score(supply_plans, 
                        demand_plans,
                        starting_inventory = 0,
                        first_date = datetime.date(2021, 7, 31), 
                        last_date = datetime.date(2021, 8, 31), 
                        decay_weight = 3.0, 
                        inv_positive_threshold = 20, 
                        inv_negative_threshold = -100):
    
    horizon = (last_date - first_date).days + 1
    # Define Inventory horizon and add starting inventory
    inventory_horizon = np.zeros(horizon, dtype=int)
    inventory_horizon = inventory_horizon + starting_inventory

    # Add Supply plans
    for i in range(len(supply_plans)):
        supply_date = datetime.datetime.fromisoformat(supply_plans[i]['startDate'][:-1]).date()
        qty = supply_plans[i]['quantity']
        diff_days = (supply_date - first_date).days
        inventory_horizon[diff_days:] = inventory_horizon[diff_days:] + qty

    for i in range(len(demand_plans)):
        demand_date = datetime.datetime.fromisoformat(demand_plans[i]['startDate'][:-1]).date()
        qty = demand_plans[i]['quantity']
        diff_days = (demand_date - first_date).days
        inventory_horizon[diff_days:] = inventory_horizon[diff_days:] - qty

    # Calculate weights
    weights = np.exp(np.arange(decay_weight, 0, -(decay_weight/horizon)))/np.exp(decay_weight)

    # Calculate penalty
    inventory_below_threshold = inventory_horizon - inv_positive_threshold
    penalties = np.zeros(horizon, dtype=int)
    neg_inv_mask = inventory_below_threshold < 0
    penalties[neg_inv_mask] = inventory_below_threshold[neg_inv_mask] 

    neg_threshold_mask = penalties < inv_negative_threshold
    penalties[neg_threshold_mask] = inv_negative_threshold

    total_penalty = np.sum(weights*-penalties)
    max_penalty = np.sum(weights*-inv_negative_threshold)
    priority_score = int(np.rint((total_penalty/max_penalty)*100))

    return priority_score


### InfoHub Connection Config

* Load your InfoHub connection parameters from `credentials.json` file
* `tenantId` is not required for establishing the connection, but is required in the GraphQL queries

In [ ]:
with open("credentials.json") as config_file:
    config = json.load(config_file)

In [ ]:
url = config['url']
headers  = config['headers']
tenantId = config['tenantId']
infohub = InfoHubConnection(url=url, tenantId=tenantId, headers=headers)

### Priority Score Config

* `timestamp`: Timestamp is needed in `upsert` operation, in ISO format.
    * In production, use the current system timestamp.
* `maxDate`: Active supply and demand plans till this date are used for priority score calculation
    * For more details, check the accompanying Notebook **AOOS-Priority-Score.ipynb**

In [ ]:
timestamp = "2021-08-03T10:37:07+0800"
maxDate = "2021-08-31 00:00:00"

### Work Queue List

* Our goal is to evaluate / re-calculate the priority score of work items in the AOOS work queue
* We need the AOOS work queue object ID to get the work items that are in progress
* To get the AOOS work queue object ID, we use the following query to get the list of work queues.

In [ ]:
workqueues = infohub.get_work_queues()
for i in range(len(workqueues)):
    print('WorkQueue: ', workqueues[i]['object']['name'])
    print('Id: ', workqueues[i]['object']['id'])
    print("--------------------------------------------------------------------------------")

### Choose `workQueueId`

* Choose the workQueueId of _Inventory approaching out of stock prioritized_

In [ ]:
workQueueId = "516dc12d-eff6-4c51-9222-7eca88a31c5e"

### Collect Work Items

* Given the work queue ID, collect all the work items

In [ ]:
work_items = []
print("Querying work items in progress...")
work_items_list = infohub.get_workitems_in_progress(workQueueId=workQueueId)
print("\tNumber of WorkItems: {}".format(len(work_items_list)))
for i in range(len(work_items_list)):
    wi = {"workItemId": work_items_list[i]['object']["id"],
          "priority": work_items_list[i]['object']["priority"],
          "inventoryId": work_items_list[i]['object']["businessObject"]["id"],
          "productId": work_items_list[i]['object']["businessObject"]["product"]["id"],
          "partNumber": work_items_list[i]['object']["businessObject"]["product"]["partNumber"],
          "locationId": work_items_list[i]['object']["businessObject"]["location"]["id"],
          "locationIdentifier": work_items_list[i]['object']["businessObject"]["location"]["locationIdentifier"],
          "starting_inventory": work_items_list[i]['object']["businessObject"]["quantity"]
    }
    work_items.append(wi)
    print("({}): Object-Id: {};".format(i, wi["workItemId"]))
    print("\tPart-Number: {}; Location-Id: {}; Priority-Score: {}".format(wi["partNumber"], wi["locationIdentifier"], wi["priority"]))
print('Done.')

### Demo: Change in Priority Score
* The priority scores are based on current supply plans and demand plans for next 30 days (`maxDate`)
* In the event of a change in supply/demand plan(s), the priority score has to change to reflect the severity of going _out of stock_
* We can test this by simulating a change in supply/demand plan (or both) and see how the priority score changes
* Steps:
    * Step 1: Choose an WorkItem
    * Step 2: Get the supply and demand plans
    * Step 3: Upsert a plan with modified quantity and/or date
        * You can repeat this for multiple plans (supply or demand or both)
    * Step 4: Calculate the new priority score and compare

#### Step 1: Choose an WorkItem
* Choose an WorkItem from the above list (by its index)

In [ ]:
k = 3
partNumber = work_items[k]["partNumber"]
locationIdentifier = work_items[k]["locationIdentifier"]
priority = work_items[k]["priority"]
workItemId = work_items[k]["workItemId"]
starting_inventory = work_items[k]["starting_inventory"]
print("Selected WorkItem: Object ID: {}".format(workItemId))
print("\t({}): Part-Number: {}; Location-Id: {}; Priority-Score: {}".format(k, partNumber, locationIdentifier, priority))

#### Step 2: Get Supply and Demand Plans
* WorkItem can be identified with its unique object ID or by its unique `partNumber` and `locationIdentifier` combination
* We have constructed all queries with `partNumber` and `locationIdentifier` combination for easy readability
    * Good practice is to use object ID as it is immune to possible changes in schema
* To calculate the priority score we need both the supply and demand plans

In [ ]:
# Get supply plans
supply_plans = []
print("Querying supply plans ...")
plan_list = infohub.get_supplyplans(partNumber=partNumber, locationIdentifier=locationIdentifier, maxDate=maxDate)
print("\tNumber of Supply Plans: {}".format(len(plan_list)))
for i in range(len(plan_list)):
    plan = {"startDate": plan_list[i]['object']["startDate"],
            "quantity": plan_list[i]['object']["quantity"],
            "id": plan_list[i]['object']["id"]
           }
    supply_plans.append(plan)

# Get demand plans
demand_plans =[]
print("Querying demand plans ...")
plan_list = infohub.get_demandplans(partNumber=partNumber, locationIdentifier=locationIdentifier, maxDate=maxDate)
print("\tNumber of Demand Plans: {}".format(len(plan_list)))
for i in range(len(plan_list)):
    plan = {"startDate": plan_list[i]['object']["startDate"],
            "quantity": plan_list[i]['object']["quantity"],
            "id": plan_list[i]['object']["id"]
           }
    demand_plans.append(plan)
print("Done.")

In [ ]:
# Print Supply and Demand Plans
print("::Supply Plans::")
for i in range(len(supply_plans)):
    print("\t({}): Object ID: {}".format(i, supply_plans[i]["id"]))
    print("\t\tStart Date: {}; Quantity: {};".format(supply_plans[i]["startDate"], supply_plans[i]["quantity"]))
print("::Demand Plans::")
for i in range(len(demand_plans)):
    print("\t({}): Object ID: {}".format(i, demand_plans[i]["id"]))
    print("\t\tStart Date: {}; Quantity: {};".format(demand_plans[i]["startDate"], demand_plans[i]["quantity"]))

#### Step 3: Upsert a plan with modified quantity and/or date

* Choose a supply/demand plan and change its quantity and/or date
* Upsert the modified plan
* You can repeat this for multiple plans (supply or demand or both)
    * Steps 3a and 3b

In [ ]:
# To demonstrate the change in the priority
changed_plans = []
new_supply_plans = copy.deepcopy(supply_plans)
new_demand_plans = copy.deepcopy(demand_plans)

**Step 3a: _Modify Supply Plan_**

In [ ]:
# Choose a supply plan to modify by its index
k = 0
id = supply_plans[k]["id"]
startDate = supply_plans[k]["startDate"]
quantity = supply_plans[k]["quantity"]
print("Selected Supply Plan ({}): Object ID: {}".format(k, id))
print("\tStart Date: {}; Quantity: {};".format(startDate, quantity))
new_startDate = startDate
new_quantity = quantity

In [ ]:
# Modify quantity and/or date (Comment out to keep it the same)
# new_startDate = "2021-08-04T00:00:00.000Z"
new_quantity = 350.0

In [ ]:
# Upsert the modified supply plan
print("Upserting modified supply plan..")
upsert_result = infohub.upsert_supplyplan(supplyPlanID=id, quantity=new_quantity, startDate=new_startDate, timestamp=timestamp)
print(upsert_result)

In [ ]:
# Update the new values in the local list
new_supply_plans[k]["quantity"] = new_quantity
new_supply_plans[k]["startDate"] = new_startDate
changed_plans.append({"planType": "supply",
                      "quantity": quantity,
                      "startDate": startDate,
                      "new_quantity": new_quantity,
                      "new_startDate": new_startDate})

**Step 3b: _Modify Demand Plan_**

In [ ]:
# Choose a demand plan to modify by its index
k = 0
id = demand_plans[k]["id"]
startDate = demand_plans[k]["startDate"]
quantity = demand_plans[k]["quantity"]
print("Selected Demand Plan ({}): Object ID: {}".format(k, id))
print("\tStart Date: {}; Quantity: {};".format(startDate, quantity))
new_startDate = startDate
new_quantity = quantity

In [ ]:
# Modify quantity and/or date (Comment out to keep it the same)
#new_startDate = "2021-08-02T00:00:00.000Z"
new_quantity = 35.0

In [ ]:
# Upsert the modified demand plan
print("Upserting modified demand plan..")
upsert_result = infohub.upsert_demandplan(demandPlanID=id, quantity=new_quantity, startDate=new_startDate, timestamp=timestamp)
print(upsert_result)

In [ ]:
# Update the new values in the local list
new_demand_plans[k]["quantity"] = new_quantity
new_demand_plans[k]["startDate"] = new_startDate
changed_plans.append({"planType": "demand",
                      "quantity": quantity,
                      "startDate": startDate,
                      "new_quantity": new_quantity,
                      "new_startDate": new_startDate})

#### Step 4: Calculate Priority Score

In [ ]:
new_priority = AOOS_priority_score(supply_plans=new_supply_plans, demand_plans=new_demand_plans, starting_inventory=starting_inventory, inv_positive_threshold=100, inv_negative_threshold=-300)
print("Updated priority score: {}".format(new_priority))


In [ ]:
# Results
print("Work Item / Inventory:")
print("----------------------")
print("\tPart Number: {}; Location: {}".format(partNumber, locationIdentifier))
print("----------------------")
print("Changed Plans:")
for i in range(len(changed_plans)):
    print("\tPlan Type: {}".format(changed_plans[i]["planType"]))
    if changed_plans[i]["quantity"] != changed_plans[i]["new_quantity"]:
        print("\t\t Quantity change: {} -> {}".format(changed_plans[i]["quantity"], changed_plans[i]["new_quantity"]))
    if changed_plans[i]["startDate"] != changed_plans[i]["new_startDate"]:
        print("\t\t Start date change: {} -> {}".format(changed_plans[i]["startDate"], changed_plans[i]["new_startDate"]))
print("---------------------------------------")
print("Priority change: {} -> {}".format(priority, new_priority))
print("---------------------------------------")

In [ ]:
# Upsert the new priority score
upsert_result = infohub.upsert_workitem_priority(workItemId=workItemId, priority=new_priority, timestamp=timestamp)
print(upsert_result)